In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
from fractions import Fraction
import warnings
warnings.filterwarnings('ignore')

In [2]:
metric = ' Inches'

range_string="""
'0 - 0.50 Inches'
 '0.51 - 1.00 Inches'
 '1.01 - 1.50 Inches'
 '1.51 - 2.00 Inches'
 '2.01 - 2.50 Inches'
 '2.51 - 3.00 Inches'
 'Greater than 3.00 Inches'
 """

In [12]:
customer_id = '134'
formatted_attribute = 'v_belt_outside_length_inches'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

buckets = """V-Belt"""

regex_pattern = r"(?i)(\d+\-?\s?\d+\/?\d*\s?in\s?overall height)|(\d+\.\d+\s?in\s?overall height)|(\b\d+\/\d+\s?in\s?overall height)|(\d+\s?in\s?overall height)"


value='%n/a%'
customer_name='%bedbathbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [13]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [14]:
# df = query_from_file(file_name='Curate_wipe/BusinessIntelligence/Platform Queries/Curation - Curation With Strategy - Products Uncurated from specific Attribute And Bucket.sql', params=params)
df = query_from_file(file_name='query/top_width.sql', params=params)
df

DatabaseError: Execution failed on sql '

SELECT
    curated_product_fields."name"  AS "attribute",
    curated_product_fields."value",
    customers."name"  AS "customers_name",
    products."external_id",
    products."name"  AS "long_desc",
    buckets."name" AS "buckets",
    CASE WHEN curation_tasks."resolution" IS NULL THEN 'standard' ELSE resolution END 
FROM public.curated_product_fields  AS curated_product_fields
LEFT JOIN public.curation_tasks  AS curation_tasks ON (curated_product_fields."curation_task_id") = (curation_tasks."id")
LEFT JOIN public.products  AS products ON (curated_product_fields."product_id") = (products."id")
LEFT JOIN public.customers  AS customers ON (curated_product_fields."customer_id") = (customers."id")
LEFT JOIN buckets.id AS buckets ON  

WHERE (curated_product_fields."name" ) = 'v_belt_outside_length_inches' 
-- AND ((curated_product_fields."value" ) NOT LIKE '%n/a%' OR (curated_product_fields."value" ) IS NULL) 
AND (buckets."name") LIKE 'V-Belt'


In [15]:
# Find a list of values to iterate
df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
df_strategy
list_values=df_strategy['value'].to_list()
list_values

['0 - 25 Inches',
 '101 - 150 Inches',
 '151 - 200 Inches',
 '201 - 250 Inches',
 '251 - 300 Inches',
 '26 - 50 Inches',
 '301 - 400 Inches',
 '401 - 500 Inches',
 '501 - 600 Inches',
 '51 - 100 Inches',
 '601 - 700 Inches',
 'Greater than 700 Inches']

In [7]:
kwp=KeywordProcessor()
print(kwp.add_keywords_from_list(list_values))
# Extract Key Words
df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

None


0      []
1      []
2      []
3      []
4      []
       ..
813    []
814    []
815    []
816    []
817    []
Name: long_desc, Length: 818, dtype: object

In [8]:
# df['matches'] = df['long_desc'].apply(lambda x: kwp.extract_keywords(x))
# df['matches'].explode().value_counts()

In [9]:
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [10]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
df.head(1)

attribute            value      customers_name  \
0  variable_speed_belt_overall_height  0 - 0.50 Inches  motionindustriesus   

  external_id  \
0    00690366   

                                                                                                               long_desc  \
0  Multi-Speed® 9804-0638 Multi-Speed Belt - 1.188 in Top Width, 54.83 in Overall Length, 0.44 in Overall Height, Rubber   

  resolution              buckets                   matches  
0      rules  Variable Speed Belt  [0.44 in Overall Height]

In [11]:
error

NameError: name 'error' is not defined

In [12]:
drop_these=df[(df['matches'].astype(str)=='[]')&(df['value'].astype(str)!='n/a')]
drop_these=drop_these['external_id']
drop_these

37     00689542
45     00689612
61     00691842
69     00692645
93     00713771
         ...   
354    10543777
355    10543778
356    10543779
357    10543780
396    00713734
Name: external_id, Length: 186, dtype: object

In [13]:
df=df[df['matches'].astype(str)!='[]']

In [14]:
def remove_txt(lst):
    assert type(lst) == list, "make sure it's a list"
    tmp = []
    for measure in lst:
        val = re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.','',measure)
        if val != '':
            tmp.append(val.replace(",",''))
    if str(tmp) == '[]':
        return ''
    else:
        return '{:,}'.format(int(max(tmp)))

In [15]:
matches=df
matches

attribute            value      customers_name  \
0    variable_speed_belt_overall_height  0 - 0.50 Inches  motionindustriesus   
1    variable_speed_belt_overall_height  0 - 0.50 Inches  motionindustriesus   
2    variable_speed_belt_overall_height  0 - 0.50 Inches  motionindustriesus   
3    variable_speed_belt_overall_height  0 - 0.50 Inches  motionindustriesus   
4    variable_speed_belt_overall_height  0 - 0.50 Inches  motionindustriesus   
..                                  ...              ...                 ...   
675  variable_speed_belt_overall_height              n/a  motionindustriesus   
676  variable_speed_belt_overall_height              n/a  motionindustriesus   
677  variable_speed_belt_overall_height              n/a  motionindustriesus   
678  variable_speed_belt_overall_height              n/a  motionindustriesus   
681  variable_speed_belt_overall_height              n/a  motionindustriesus   

    external_id  \
0      00690366   
1      00690887   
2      00690958   
3      00691028   
4      00691182   
..          ...   
675    00729666   
676    00729673   
677    00729681   
678    00729688   
681    00729843   

                                                                                                                      long_desc  \
0         Multi-Speed® 9804-0638 Multi-Speed Belt - 1.188 in Top Width, 54.83 in Overall Length, 0.44 in Overall Height, Rubber   
1           Multi-Speed® 9804-0479 Multi-Speed Belt - 1-7/16 in Top Width, 38 in Overall Length, 0.40 in Overall Height, Rubber   
2        Multi-Speed® 9800-0069 Multi-Speed Belt - 1.569 in Top Width, 30.65 in Overall Length, 0.318 in Overall Height, Rubber   
3        Multi-Speed® 9800-0260 Multi-Speed Belt - 1.579 in Top Width, 62.49 in Overall Length, 0.467 in Overall Height, Rubber   
4        Multi-Speed® 9800-0904 Multi-Speed Belt - 1.563 in Top Width, 149.74 in Overall Length, 0.47 in Overall Height, Rubber   
..                                                                                                                          ...   
675     Multi-Speed® 9804-0764 Multi-Speed Belt - 1-3/16 in Top Width, 56.9 in Overall Length, 1-3/16 in Overall Height, Rubber   
676  Multi-Speed® 9804-0905 Variable Speed Belt - 1-7/16 in Top Width, 39.3 in Overall Length, 1-7/16 in Overall Height, Rubber   
677       Multi-Speed® 9804-0726 Multi-Speed Belt - 1-7/16 in Top Width, 47 in Overall Length, 1-7/16 in Overall Height, Rubber   
678     Multi-Speed® 9800-0728 Multi-Speed Belt - 1-7/16 in Top Width, 34.6 in Overall Length, 1-7/16 in Overall Height, Rubber   
681       Multi-Speed® 9804-0749 Multi-Speed Belt - 1-5/16 in Top Width, 38 in Overall Length, 1-5/16 in Overall Height, Rubber   

    resolution              buckets                     matches  
0        rules  Variable Speed Belt    [0.44 in Overall Height]  
1        rules  Variable Speed Belt    [0.40 in Overall Height]  
2        rules  Variable Speed Belt   [0.318 in Overall Height]  
3        rules  Variable Speed Belt   [0.467 in Overall Height]  
4        rules  Variable Speed Belt    [0.47 in Overall Height]  
..         ...                  ...                         ...  
675       bulk  Variable Speed Belt  [1-3/16 in Overall Height]  
676       bulk  Variable Speed Belt  [1-7/16 in Overall Height]  
677       bulk  Variable Speed Belt  [1-7/16 in Overall Height]  
678       bulk  Variable Speed Belt  [1-7/16 in Overall Height]  
681       bulk  Variable Speed Belt  [1-5/16 in Overall Height]  

[462 rows x 8 columns]

In [16]:
# matches['matches']=matches['matches'].str.get(0)

In [17]:
matches['matches'] = matches['matches'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.",'',str(measure)))
# matches['matches'] = matches['matches'].apply(lambda quote: quote.replace("'", ''))

In [18]:
matches

attribute            value      customers_name  \
0    variable_speed_belt_overall_height  0 - 0.50 Inches  motionindustriesus   
1    variable_speed_belt_overall_height  0 - 0.50 Inches  motionindustriesus   
2    variable_speed_belt_overall_height  0 - 0.50 Inches  motionindustriesus   
3    variable_speed_belt_overall_height  0 - 0.50 Inches  motionindustriesus   
4    variable_speed_belt_overall_height  0 - 0.50 Inches  motionindustriesus   
..                                  ...              ...                 ...   
675  variable_speed_belt_overall_height              n/a  motionindustriesus   
676  variable_speed_belt_overall_height              n/a  motionindustriesus   
677  variable_speed_belt_overall_height              n/a  motionindustriesus   
678  variable_speed_belt_overall_height              n/a  motionindustriesus   
681  variable_speed_belt_overall_height              n/a  motionindustriesus   

    external_id  \
0      00690366   
1      00690887   
2      00690958   
3      00691028   
4      00691182   
..          ...   
675    00729666   
676    00729673   
677    00729681   
678    00729688   
681    00729843   

                                                                                                                      long_desc  \
0         Multi-Speed® 9804-0638 Multi-Speed Belt - 1.188 in Top Width, 54.83 in Overall Length, 0.44 in Overall Height, Rubber   
1           Multi-Speed® 9804-0479 Multi-Speed Belt - 1-7/16 in Top Width, 38 in Overall Length, 0.40 in Overall Height, Rubber   
2        Multi-Speed® 9800-0069 Multi-Speed Belt - 1.569 in Top Width, 30.65 in Overall Length, 0.318 in Overall Height, Rubber   
3        Multi-Speed® 9800-0260 Multi-Speed Belt - 1.579 in Top Width, 62.49 in Overall Length, 0.467 in Overall Height, Rubber   
4        Multi-Speed® 9800-0904 Multi-Speed Belt - 1.563 in Top Width, 149.74 in Overall Length, 0.47 in Overall Height, Rubber   
..                                                                                                                          ...   
675     Multi-Speed® 9804-0764 Multi-Speed Belt - 1-3/16 in Top Width, 56.9 in Overall Length, 1-3/16 in Overall Height, Rubber   
676  Multi-Speed® 9804-0905 Variable Speed Belt - 1-7/16 in Top Width, 39.3 in Overall Length, 1-7/16 in Overall Height, Rubber   
677       Multi-Speed® 9804-0726 Multi-Speed Belt - 1-7/16 in Top Width, 47 in Overall Length, 1-7/16 in Overall Height, Rubber   
678     Multi-Speed® 9800-0728 Multi-Speed Belt - 1-7/16 in Top Width, 34.6 in Overall Length, 1-7/16 in Overall Height, Rubber   
681       Multi-Speed® 9804-0749 Multi-Speed Belt - 1-5/16 in Top Width, 38 in Overall Length, 1-5/16 in Overall Height, Rubber   

    resolution              buckets   matches  
0        rules  Variable Speed Belt    '0.44'  
1        rules  Variable Speed Belt    '0.40'  
2        rules  Variable Speed Belt   '0.318'  
3        rules  Variable Speed Belt   '0.467'  
4        rules  Variable Speed Belt    '0.47'  
..         ...                  ...       ...  
675       bulk  Variable Speed Belt  '1-3/16'  
676       bulk  Variable Speed Belt  '1-7/16'  
677       bulk  Variable Speed Belt  '1-7/16'  
678       bulk  Variable Speed Belt  '1-7/16'  
681       bulk  Variable Speed Belt  '1-5/16'  

[462 rows x 8 columns]

In [22]:
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers

In [23]:
matches['matches'] = matches['matches'].apply(lambda x: remove_fractions(x))
matches

attribute            value      customers_name  \
0    variable_speed_belt_overall_height  0 - 0.50 Inches  motionindustriesus   
1    variable_speed_belt_overall_height  0 - 0.50 Inches  motionindustriesus   
2    variable_speed_belt_overall_height  0 - 0.50 Inches  motionindustriesus   
3    variable_speed_belt_overall_height  0 - 0.50 Inches  motionindustriesus   
4    variable_speed_belt_overall_height  0 - 0.50 Inches  motionindustriesus   
..                                  ...              ...                 ...   
675  variable_speed_belt_overall_height              n/a  motionindustriesus   
676  variable_speed_belt_overall_height              n/a  motionindustriesus   
677  variable_speed_belt_overall_height              n/a  motionindustriesus   
678  variable_speed_belt_overall_height              n/a  motionindustriesus   
681  variable_speed_belt_overall_height              n/a  motionindustriesus   

    external_id  \
0      00690366   
1      00690887   
2      00690958   
3      00691028   
4      00691182   
..          ...   
675    00729666   
676    00729673   
677    00729681   
678    00729688   
681    00729843   

                                                                                                                      long_desc  \
0         Multi-Speed® 9804-0638 Multi-Speed Belt - 1.188 in Top Width, 54.83 in Overall Length, 0.44 in Overall Height, Rubber   
1           Multi-Speed® 9804-0479 Multi-Speed Belt - 1-7/16 in Top Width, 38 in Overall Length, 0.40 in Overall Height, Rubber   
2        Multi-Speed® 9800-0069 Multi-Speed Belt - 1.569 in Top Width, 30.65 in Overall Length, 0.318 in Overall Height, Rubber   
3        Multi-Speed® 9800-0260 Multi-Speed Belt - 1.579 in Top Width, 62.49 in Overall Length, 0.467 in Overall Height, Rubber   
4        Multi-Speed® 9800-0904 Multi-Speed Belt - 1.563 in Top Width, 149.74 in Overall Length, 0.47 in Overall Height, Rubber   
..                                                                                                                          ...   
675     Multi-Speed® 9804-0764 Multi-Speed Belt - 1-3/16 in Top Width, 56.9 in Overall Length, 1-3/16 in Overall Height, Rubber   
676  Multi-Speed® 9804-0905 Variable Speed Belt - 1-7/16 in Top Width, 39.3 in Overall Length, 1-7/16 in Overall Height, Rubber   
677       Multi-Speed® 9804-0726 Multi-Speed Belt - 1-7/16 in Top Width, 47 in Overall Length, 1-7/16 in Overall Height, Rubber   
678     Multi-Speed® 9800-0728 Multi-Speed Belt - 1-7/16 in Top Width, 34.6 in Overall Length, 1-7/16 in Overall Height, Rubber   
681       Multi-Speed® 9804-0749 Multi-Speed Belt - 1-5/16 in Top Width, 38 in Overall Length, 1-5/16 in Overall Height, Rubber   

    resolution              buckets  matches  
0        rules  Variable Speed Belt   '0.44'  
1        rules  Variable Speed Belt   '0.40'  
2        rules  Variable Speed Belt  '0.318'  
3        rules  Variable Speed Belt  '0.467'  
4        rules  Variable Speed Belt   '0.47'  
..         ...                  ...      ...  
675       bulk  Variable Speed Belt   1.1875  
676       bulk  Variable Speed Belt   1.4375  
677       bulk  Variable Speed Belt   1.4375  
678       bulk  Variable Speed Belt   1.4375  
681       bulk  Variable Speed Belt   1.3125  

[462 rows x 8 columns]

In [28]:
def remove_fraction(numbers):
    if numbers.find('/') > -1:
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
        return str(tmp)
    else:
        return numbers

In [29]:
#matches['matches'].replace(r'\-','',regex=True,inplace=True)
matches['matches']=matches['matches'].apply(lambda quote: quote.replace("'", ''))
matches['matches']=matches['matches'].apply(lambda x: remove_fraction(x))

In [33]:
matches['matches'].explode().value_counts()[50:100]

0.3100    1
0.65      1
0.886     1
0.318     1
0.836     1
0.39      1
0.501     1
0.689     1
0.646     1
0.647     1
0.51      1
0.69      1
0.467     1
0.808     1
0.504     1
0.869     1
0.451     1
0.5       1
0.40      1
3.875     1
0.8125    1
0.35      1
3.1875    1
0.57      1
0.54      1
2.3125    1
Name: matches, dtype: int64

In [34]:
# matches['new']=matches['matches'].apply(lambda x: kwp.extract_keywords(x))
matches

attribute            value      customers_name  \
0    variable_speed_belt_overall_height  0 - 0.50 Inches  motionindustriesus   
1    variable_speed_belt_overall_height  0 - 0.50 Inches  motionindustriesus   
2    variable_speed_belt_overall_height  0 - 0.50 Inches  motionindustriesus   
3    variable_speed_belt_overall_height  0 - 0.50 Inches  motionindustriesus   
4    variable_speed_belt_overall_height  0 - 0.50 Inches  motionindustriesus   
..                                  ...              ...                 ...   
675  variable_speed_belt_overall_height              n/a  motionindustriesus   
676  variable_speed_belt_overall_height              n/a  motionindustriesus   
677  variable_speed_belt_overall_height              n/a  motionindustriesus   
678  variable_speed_belt_overall_height              n/a  motionindustriesus   
681  variable_speed_belt_overall_height              n/a  motionindustriesus   

    external_id  \
0      00690366   
1      00690887   
2      00690958   
3      00691028   
4      00691182   
..          ...   
675    00729666   
676    00729673   
677    00729681   
678    00729688   
681    00729843   

                                                                                                                      long_desc  \
0         Multi-Speed® 9804-0638 Multi-Speed Belt - 1.188 in Top Width, 54.83 in Overall Length, 0.44 in Overall Height, Rubber   
1           Multi-Speed® 9804-0479 Multi-Speed Belt - 1-7/16 in Top Width, 38 in Overall Length, 0.40 in Overall Height, Rubber   
2        Multi-Speed® 9800-0069 Multi-Speed Belt - 1.569 in Top Width, 30.65 in Overall Length, 0.318 in Overall Height, Rubber   
3        Multi-Speed® 9800-0260 Multi-Speed Belt - 1.579 in Top Width, 62.49 in Overall Length, 0.467 in Overall Height, Rubber   
4        Multi-Speed® 9800-0904 Multi-Speed Belt - 1.563 in Top Width, 149.74 in Overall Length, 0.47 in Overall Height, Rubber   
..                                                                                                                          ...   
675     Multi-Speed® 9804-0764 Multi-Speed Belt - 1-3/16 in Top Width, 56.9 in Overall Length, 1-3/16 in Overall Height, Rubber   
676  Multi-Speed® 9804-0905 Variable Speed Belt - 1-7/16 in Top Width, 39.3 in Overall Length, 1-7/16 in Overall Height, Rubber   
677       Multi-Speed® 9804-0726 Multi-Speed Belt - 1-7/16 in Top Width, 47 in Overall Length, 1-7/16 in Overall Height, Rubber   
678     Multi-Speed® 9800-0728 Multi-Speed Belt - 1-7/16 in Top Width, 34.6 in Overall Length, 1-7/16 in Overall Height, Rubber   
681       Multi-Speed® 9804-0749 Multi-Speed Belt - 1-5/16 in Top Width, 38 in Overall Length, 1-5/16 in Overall Height, Rubber   

    resolution              buckets matches  
0        rules  Variable Speed Belt    0.44  
1        rules  Variable Speed Belt    0.40  
2        rules  Variable Speed Belt   0.318  
3        rules  Variable Speed Belt   0.467  
4        rules  Variable Speed Belt    0.47  
..         ...                  ...     ...  
675       bulk  Variable Speed Belt  1.1875  
676       bulk  Variable Speed Belt  1.4375  
677       bulk  Variable Speed Belt  1.4375  
678       bulk  Variable Speed Belt  1.4375  
681       bulk  Variable Speed Belt  1.3125  

[462 rows x 8 columns]

In [35]:
string = range_string.replace(metric, '')
lst = string.split('\n')
builder_base = f'''
def range_app(num):
'''
for val in lst:
    if val.find(' - ') >-1:
        original_val = val
        lower, upper = val.split(' - ')
        inner_f = f'''
    if num >= {lower} and num <= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
    if val.find('Greater than ') >-1:
        original_val = val
        upper = val.replace('Greater than ','')
        inner_f = f'''
    if num >= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
final_else = '''
    else:
        return r"n/a"
'''
exec(builder_base + final_else)
print( builder_base + final_else)


def range_app(num):

    if num >= '0 and num <= 0.50':
        return "'0 - 0.50' Inches"
        
    if num >=  '0.51 and num <= 1.00':
        return " '0.51 - 1.00' Inches"
        
    if num >=  '1.01 and num <= 1.50':
        return " '1.01 - 1.50' Inches"
        
    if num >=  '1.51 and num <= 2.00':
        return " '1.51 - 2.00' Inches"
        
    if num >=  '2.01 and num <= 2.50':
        return " '2.01 - 2.50' Inches"
        
    if num >=  '2.51 and num <= 3.00':
        return " '2.51 - 3.00' Inches"
        
    if num >=  '3.00':
        return " 'Greater than 3.00' Inches"
        
    else:
        return r"n/a"



In [36]:
matches[curation_col] = matches['matches'].apply(range_app)
matches

attribute            value      customers_name  \
0    variable_speed_belt_overall_height  0 - 0.50 Inches  motionindustriesus   
1    variable_speed_belt_overall_height  0 - 0.50 Inches  motionindustriesus   
2    variable_speed_belt_overall_height  0 - 0.50 Inches  motionindustriesus   
3    variable_speed_belt_overall_height  0 - 0.50 Inches  motionindustriesus   
4    variable_speed_belt_overall_height  0 - 0.50 Inches  motionindustriesus   
..                                  ...              ...                 ...   
675  variable_speed_belt_overall_height              n/a  motionindustriesus   
676  variable_speed_belt_overall_height              n/a  motionindustriesus   
677  variable_speed_belt_overall_height              n/a  motionindustriesus   
678  variable_speed_belt_overall_height              n/a  motionindustriesus   
681  variable_speed_belt_overall_height              n/a  motionindustriesus   

    external_id  \
0      00690366   
1      00690887   
2      00690958   
3      00691028   
4      00691182   
..          ...   
675    00729666   
676    00729673   
677    00729681   
678    00729688   
681    00729843   

                                                                                                                      long_desc  \
0         Multi-Speed® 9804-0638 Multi-Speed Belt - 1.188 in Top Width, 54.83 in Overall Length, 0.44 in Overall Height, Rubber   
1           Multi-Speed® 9804-0479 Multi-Speed Belt - 1-7/16 in Top Width, 38 in Overall Length, 0.40 in Overall Height, Rubber   
2        Multi-Speed® 9800-0069 Multi-Speed Belt - 1.569 in Top Width, 30.65 in Overall Length, 0.318 in Overall Height, Rubber   
3        Multi-Speed® 9800-0260 Multi-Speed Belt - 1.579 in Top Width, 62.49 in Overall Length, 0.467 in Overall Height, Rubber   
4        Multi-Speed® 9800-0904 Multi-Speed Belt - 1.563 in Top Width, 149.74 in Overall Length, 0.47 in Overall Height, Rubber   
..                                                                                                                          ...   
675     Multi-Speed® 9804-0764 Multi-Speed Belt - 1-3/16 in Top Width, 56.9 in Overall Length, 1-3/16 in Overall Height, Rubber   
676  Multi-Speed® 9804-0905 Variable Speed Belt - 1-7/16 in Top Width, 39.3 in Overall Length, 1-7/16 in Overall Height, Rubber   
677       Multi-Speed® 9804-0726 Multi-Speed Belt - 1-7/16 in Top Width, 47 in Overall Length, 1-7/16 in Overall Height, Rubber   
678     Multi-Speed® 9800-0728 Multi-Speed Belt - 1-7/16 in Top Width, 34.6 in Overall Length, 1-7/16 in Overall Height, Rubber   
681       Multi-Speed® 9804-0749 Multi-Speed Belt - 1-5/16 in Top Width, 38 in Overall Length, 1-5/16 in Overall Height, Rubber   

    resolution              buckets matches  \
0        rules  Variable Speed Belt    0.44   
1        rules  Variable Speed Belt    0.40   
2        rules  Variable Speed Belt   0.318   
3        rules  Variable Speed Belt   0.467   
4        rules  Variable Speed Belt    0.47   
..         ...                  ...     ...   
675       bulk  Variable Speed Belt  1.1875   
676       bulk  Variable Speed Belt  1.4375   
677       bulk  Variable Speed Belt  1.4375   
678       bulk  Variable Speed Belt  1.4375   
681       bulk  Variable Speed Belt  1.3125   

    Q:variable_speed_belt_overall_height  
0                      '0 - 0.50' Inches  
1                      '0 - 0.50' Inches  
2                      '0 - 0.50' Inches  
3                      '0 - 0.50' Inches  
4                      '0 - 0.50' Inches  
..                                   ...  
675                    '0 - 0.50' Inches  
676                    '0 - 0.50' Inches  
677                    '0 - 0.50' Inches  
678                    '0 - 0.50' Inches  
681                    '0 - 0.50' Inches  

[462 rows x 9 columns]

In [39]:
matches[100:150]

attribute               value  \
149  variable_speed_belt_overall_height  1.01 - 1.50 Inches   
150  variable_speed_belt_overall_height  1.01 - 1.50 Inches   
151  variable_speed_belt_overall_height  1.01 - 1.50 Inches   
152  variable_speed_belt_overall_height  1.01 - 1.50 Inches   
153  variable_speed_belt_overall_height  1.01 - 1.50 Inches   
154  variable_speed_belt_overall_height  1.01 - 1.50 Inches   
155  variable_speed_belt_overall_height  1.01 - 1.50 Inches   
156  variable_speed_belt_overall_height  1.01 - 1.50 Inches   
157  variable_speed_belt_overall_height  1.01 - 1.50 Inches   
158  variable_speed_belt_overall_height  1.01 - 1.50 Inches   
159  variable_speed_belt_overall_height  1.01 - 1.50 Inches   
160  variable_speed_belt_overall_height  1.01 - 1.50 Inches   
161  variable_speed_belt_overall_height  1.01 - 1.50 Inches   
162  variable_speed_belt_overall_height  1.01 - 1.50 Inches   
163  variable_speed_belt_overall_height  1.01 - 1.50 Inches   
164  variable_speed_belt_overall_height  1.01 - 1.50 Inches   
165  variable_speed_belt_overall_height  1.01 - 1.50 Inches   
199  variable_speed_belt_overall_height  1.51 - 2.00 Inches   
200  variable_speed_belt_overall_height  1.51 - 2.00 Inches   
201  variable_speed_belt_overall_height  1.51 - 2.00 Inches   
202  variable_speed_belt_overall_height  1.51 - 2.00 Inches   
203  variable_speed_belt_overall_height  1.51 - 2.00 Inches   
204  variable_speed_belt_overall_height  1.51 - 2.00 Inches   
205  variable_speed_belt_overall_height  1.51 - 2.00 Inches   
206  variable_speed_belt_overall_height  1.51 - 2.00 Inches   
207  variable_speed_belt_overall_height  1.51 - 2.00 Inches   
208  variable_speed_belt_overall_height  1.51 - 2.00 Inches   
209  variable_speed_belt_overall_height  1.51 - 2.00 Inches   
210  variable_speed_belt_overall_height  1.51 - 2.00 Inches   
211  variable_speed_belt_overall_height  1.51 - 2.00 Inches   
212  variable_speed_belt_overall_height  1.51 - 2.00 Inches   
213  variable_speed_belt_overall_height  1.51 - 2.00 Inches   
214  variable_speed_belt_overall_height  1.51 - 2.00 Inches   
215  variable_speed_belt_overall_height  1.51 - 2.00 Inches   
216  variable_speed_belt_overall_height  1.51 - 2.00 Inches   
217  variable_speed_belt_overall_height  1.51 - 2.00 Inches   
218  variable_speed_belt_overall_height  1.51 - 2.00 Inches   
220  variable_speed_belt_overall_height  1.51 - 2.00 Inches   
221  variable_speed_belt_overall_height  1.51 - 2.00 Inches   
222  variable_speed_belt_overall_height  1.51 - 2.00 Inches   
223  variable_speed_belt_overall_height  1.51 - 2.00 Inches   
224  variable_speed_belt_overall_height  1.51 - 2.00 Inches   
225  variable_speed_belt_overall_height  1.51 - 2.00 Inches   
226  variable_speed_belt_overall_height  1.51 - 2.00 Inches   
227  variable_speed_belt_overall_height  1.51 - 2.00 Inches   
228  variable_speed_belt_overall_height  1.51 - 2.00 Inches   
229  variable_speed_belt_overall_height  1.51 - 2.00 Inches   
230  variable_speed_belt_overall_height  1.51 - 2.00 Inches   
231  variable_speed_belt_overall_height  1.51 - 2.00 Inches   
232  variable_speed_belt_overall_height  1.51 - 2.00 Inches   

         customers_name external_id  \
149  motionindustriesus    00690510   
150  motionindustriesus    00690516   
151  motionindustriesus    00690522   
152  motionindustriesus    00690533   
153  motionindustriesus    00690911   
154  motionindustriesus    00690943   
155  motionindustriesus    00690952   
156  motionindustriesus    00691713   
157  motionindustriesus    00691718   
158  motionindustriesus    00691724   
159  motionindustriesus    00713043   
160  motionindustriesus    00713069   
161  motionindustriesus    00713121   
162  motionindustriesus    00713127   
163  motionindustriesus    00713365   
164  motionindustriesus    00713453   
165  motionindustriesus    00724042   
199  motionindustriesus    00690966   
200  motionindustriesus    00690976   
201  motionindustriesus    00690985 

In [ ]:
matches[curation_col] = matches['matches']
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
matches

In [ ]:
## KWP
# matches[curation_col]=matches['matches']
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
# matches

In [ ]:
error----Switch Max and Min when doing length and width

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',matches)

# send to the folder for upload

In [ ]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload/Shawn'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute} - match upload {buckets}.csv',index=False)

In [ ]:
looks_good('Bed Bath & Beyond', attribute, df, matches)